This script reads a shapefile of US Counties provided by the NWS (https://www.weather.gov/gis/Counties), extracts the longitude and latitude of the centroid of each county, and rounds to the nearest 0.25 degree (to be compatible with the ERA5 data set).

In [73]:
import geopandas
import pandas as pd

#Load the shapefile ../Data/NWS_US_Counties_Shapefiles/c_05mr24.shp
counties = geopandas.read_file('../Data/NWS_US_Counties_Shapefiles/c_05mr24.shp')

#Remove the geometry, and time zone variables from counties
counties = counties.drop(columns=['geometry', 'TIME_ZONE'])

#Restrict data to the continental US: Remove all entries in counties where STATE is AS, PR, VI, PW, MH, MP, GU, HI, AK, or FM
counties = counties[~counties['STATE'].isin(['AS', 'PR', 'VI', 'PW', 'MH', 'MP', 'GU', 'FM', 'HI', 'AK'])]

#Drop the columns CWA and FE_AREA
counties = counties.drop(columns=['CWA', 'FE_AREA'])

#Convert FIPS in counties to float64 (to facilitate merging with other data sets)
counties['FIPS'] = counties['FIPS'].astype(float)

#Round LON and LAT to the nearest quarter-degree to facilitate merging with ERA5 data
counties['LON'] = (counties['LON'] * 4).round() / 4
counties['LAT'] = (counties['LAT'] * 4).round() / 4

#Create a geometry column (for use with geopandas) from the longitude and latitude
counties['geometry'] = geopandas.points_from_xy(counties['LON'], counties['LAT'])

#Export counties as a csv file
counties.to_csv('../Data/counties_centroids.csv', index=False)